# How to stream LLM tokens from your graph
- https://langchain-ai.github.io/langgraph/how-tos/streaming-tokens/
- `graph.stream(..., stream_mode="messages")`

In [1]:
from typing import TypedDict
from langgraph.graph import START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI


joke_model = ChatOpenAI(model="gpt-4o-mini", tags=["joke"])
poem_model = ChatOpenAI(model="gpt-4o-mini", tags=["poem"])


class State(TypedDict):
    topic: str
    joke: str
    poem: str


async def call_model(state, config):
    topic = state["topic"]
    print("Writing joke...")
    joke_response = await joke_model.ainvoke(
        [{"role": "user", "content": f"Write a joke about {topic}"}],
        config
    )
    print("\n\nWriting poem...")
    poem_response = await poem_model.ainvoke(
        [{"role": "user", "content": f"Write a short poem about {topic}"}],
        config
    )
    return {"joke": joke_response.content, "poem": poem_response.content}


graph = StateGraph(State).add_node(call_model).add_edge(START, "call_model").compile()

In [5]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="messages"
):
    if msg.content:
        print(msg.content, end="|", flush=True)

Writing joke...
Why| did| the| cat| sit| on| the| computer|?| 

|Because| it| wanted| to| keep| an| eye| on| the| mouse|!|

Writing poem...
In| shadows| sleek|,| where| silence| p|urr|s|,|  
|Soft| whispers| dance| in| twilight|'s| bl|urs|.|  
|With| eyes| like| moons| that| softly| gle|am|,|  
|They| float| through| dreams|,| a| dusk|-lit| dream|.|  

|Wh|isk|ers| twitch| at| secrets| held|,|  
|In| sun|lit| spots|,| their| warmth| rep|elled|.|  
|A| gentle| leap|,| a| playful| dart|,|  
|Each| cat| a| keeper| of| the| heart|.|  

|With| velvet| paws| on| padded| floors|,|  
|They| claim| the| night|,| then| seek| the| doors|.|  
|In| every| curl| and| playful| sw|at|,|  
|A| world| of| magic|,| they| have| sought|.|  

|So| here|’s| to| cats|,| both| wild| and| wise|,|  
|With| every| p|urr|,| they| mesmer|ize|.|  
|In| their| quiet| grace|,| we| find| our| way|,|  
|Our| furry| friends|,| by| night| and| day|.|  |

In [6]:
metadata

{'langgraph_step': 1,
 'langgraph_node': 'call_model',
 'langgraph_triggers': ['start:call_model'],
 'langgraph_path': ('__pregel_pull', 'call_model'),
 'langgraph_checkpoint_ns': 'call_model:0ebb5c41-ff2a-2c71-c231-f6e9c44d53cb',
 'checkpoint_ns': 'call_model:0ebb5c41-ff2a-2c71-c231-f6e9c44d53cb',
 'ls_provider': 'openai',
 'ls_model_name': 'gpt-4o-mini',
 'ls_model_type': 'chat',
 'ls_temperature': None,
 'tags': ['poem']}

### Filter to specific LLM invocation

In [7]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="messages"
):
    if msg.content and "joke" in metadata.get("tags", []):
        print(msg.content, end="|", flush=True)

Writing joke...
Why| was| the| cat| sitting| on| the| computer|?

|Because| it| wanted| to| keep| an| eye| on| the| mouse|!|

Writing poem...


### Example without LangChain

In [8]:
from openai import AsyncOpenAI

openai_client = AsyncOpenAI()
model_name = "gpt-4o-mini"

async def stream_tokens(model_name: str, messages: list[dict]):
    response = await openai_client.chat.completions.create(
        messages=messages, model=model_name, stream=True
    )
    role = None

    async for chunk in response:
        delta = chunk.choices[0].delta
        if delta.role is not None:
            role = delta.role

        if delta.content:
            yield {"role": role, "content": delta.content}


async def call_model(state, config, writer):
    topic = state["topic"]
    joke = ""
    poem = ""

    print("Writing joke...")
    async for msg_chunk in stream_tokens(
        model_name, [{"role": "user", "content": f"Write a joke about {topic}"}]
    ):
        joke += msg_chunk["content"]
        metadata = {**config["metadata"], "tags": ["joke"]}
        chunk_to_stream = (msg_chunk, metadata)
        writer(chunk_to_stream)

    print("\n\nWriting poem...")
    async for msg_chunk in stream_tokens(
        model_name, [{"role": "user", "content": f"Write a short poem about {topic}"}]
    ):
        poem += msg_chunk["content"]
        metadata = {**config["metadata"], "tags": ["poem"]}
        chunk_to_stream = (msg_chunk, metadata)
        writer(chunk_to_stream)


    return {"joke": joke, "poem": poem}


graph = StateGraph(State).add_node(call_model).add_edge(START, "call_model").compile()

In [10]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="custom"
):
    print(msg["content"], end="|", flush=True)

Writing joke...
Why| did| the| cat| sit| on| the| computer|?

|Because| it| wanted| to| keep| an| eye| on| the| mouse|!|

Writing poem...
In| shadows| soft|,| they| weave| and| glide|,|  
|Wh|isk|ered| whispers|,| playful| pride|.|  
|With| emerald| eyes| like| twilight| skies|,|  
|In| every| p|ounce|,| a| secret| lies|.|  

|N|im|ble| dancers|,| sleek| and| spr|y|,|  
|Ch|asing| dreams| as| moon|be|ams| fly|.|  
|Cur|led| in| warmth|,| in| sun|lit| spots|,|  
|In| their| still|ness|,| time| forget|s|.|  

|M|yster|ious| minds|,| both| wise| and| sly|,|  
|Comp|an|ions| close|,| yet| free| to| fly|.|  
|In| rhythmic| p|urr|s|,| they| claim| their| throne|,|  
|In| every| heart|,| a| cat| makes| home|.|  |

In [11]:
metadata

{'langgraph_step': 1,
 'langgraph_node': 'call_model',
 'langgraph_triggers': ['start:call_model'],
 'langgraph_path': ('__pregel_pull', 'call_model'),
 'langgraph_checkpoint_ns': 'call_model:6b812dc0-390c-c6db-5435-7a23ffa19f95',
 'tags': ['poem']}

In [12]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="custom",
):
    if "poem" in metadata.get("tags", []):
        print(msg["content"], end="|", flush=True)

Writing joke...


Writing poem...
In| shadows| soft|,| where| whispers| dwell|,|  
|A| feline| grace|,| a| myst|ic| spell|.|  
|With| emerald| eyes| and| nim|ble| paws|,|  
|They| weave| through| dreams|,| without| a| pause|.|  

|On| sun|be|ams| bright|,| they| stretch| and| y|awn|,|  
|A| sym|phony| of| p|urr|s| at| dawn|.|  
|With| playful| leaps| and| silent| tread|,|  
|They| reign| the| night|,| their| kingdom| spread|.|  

|Oh|,| curious| hearts|,| with| spirits| free|,|  
|In| every| p|ounce|,| a| glimpse| of| g|lee|.|  
|For| in| their| gaze|,| the| world's| a| stage|,|  
|Each| whisk|ered| tale|,| a| timeless| page|.|  |